In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
import torchvision.transforms as transforms



In [2]:
js_dataset = JSdataset('/data/js_detection/dataset',transform=transforms.Compose([
                                                                    transforms.ToTensor(),
                                                                    transforms.Resize((224,224))
                                                                    ]))

In [3]:
loader = DataLoader(js_dataset,batch_size=10,shuffle=True)

In [4]:
class Face_detector(nn.Module):
    def __init__(self):
        
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3,8,kernel_size=3,padding=1),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            nn.Conv2d(8,16,kernel_size=3,padding=1),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            nn.Conv2d(16,32,kernel_size=3,padding=1),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            nn.Conv2d(32,64,kernel_size=3,padding=1),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            nn.Conv2d(64,128,kernel_size=3,padding=1),
            nn.MaxPool2d(2,2),
            nn.ReLU(),
            nn.Conv2d(128,128,kernel_size=7),
            nn.ReLU()
        )
        
        self.fc1 = nn.Linear(128,128)
        self.fc2 = nn.Linear(128,2)
        
        self.relu = nn.ReLU()
        
        

    def forward(self,x):
        x=self.model(x)
        x = x.view(x.shape[0],-1)
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x
    
    def __repr__(self) -> str:
        return "FaceDetector()"
    

In [22]:
import torch
from model import FaceDetector
from dataset import JSdataset
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn

import os
import tqdm

import time
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FaceDetector()
model.to(device)

optimizer = optim.Adagrad(model.parameters(), lr=1e-3)
criterion = torch.nn.BCELoss()

In [32]:
print(model)
print(f" number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

FaceDetector()
 number of parameters: 917969


In [51]:
images,labels=next(iter(loader))



images = images.to(device)
labels = labels.type(torch.FloatTensor)
labels = labels.to(device)

loss = 0

#initialize gradient
optimizer.zero_grad()
#forward propagation
output = model(images)
#calculate loss
loss = criterion(output.squeeze(1),labels)
#backward proopagation
loss.backward()
#update parameters
optimizer.step()


loss

tensor(0.6614, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

In [52]:
labels.type()

'torch.cuda.FloatTensor'

In [53]:
output.type()

'torch.cuda.FloatTensor'

In [54]:
output

tensor([[0.5262],
        [0.5248],
        [0.5274],
        [0.5273],
        [0.5277],
        [0.5252],
        [0.5244],
        [0.5274],
        [0.5260],
        [0.5276]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [56]:
labels

tensor([1., 0., 1., 1., 1., 1., 0., 1., 1., 1.], device='cuda:0')

In [57]:
import cv2



In [58]:
images.shape

torch.Size([10, 3, 224, 224])

In [63]:
img =cv2.imread('/data/js_detection/dataset/pos/image00001.png')

In [65]:
img.shape

(480, 640, 3)

In [66]:
transform =transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224,224)),
        transforms.Normalize((0.5244, 0.5401, 0.5397),(0.2569, 0.2638, 0.2730))
    ])

In [67]:
img = transform(img)

In [69]:
img.shape

torch.Size([3, 224, 224])